# ML in MT

## Photon Time Propagation

**Idea:**

1) Set up NN that takes in `TDC` info of only parallel trays

2) Target data would be the `diffTDC` in their respective orthogonal trays

3) Initially, Training and Testing data would be samples from calibration

    a) Later, we can use data from other configurations for both
    
4) Objective Function: `abs(predicted_diffTDC - actual_diffTDC)`


### Train the Network

In [43]:
def TrainNN(df_train,df_test, targetList, inputList,emissions,epoch=1):
    
    target = [df_train[targetList[0]].to_numpy(), df_train[targetList[1]].to_numpy()]
    df_train.drop(columns=targetList,inplace=True)
 
    df_train = df_train[inputList]
    df_test = df_test[inputList]
    
    emissions = "{}_epoch_{}".format(emissions,epoch)

    tensorboard_callbacks = TensorBoard(log_dir=emissions + "/logs",
                                            histogram_freq=1)

    NN_model = Make_network(df_train.shape[1])

    checkpoint_name = emissions + '/Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
    checkpoint = ModelCheckpoint(checkpoint_name,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 mode='auto')

    callbacks_list = [checkpoint, tensorboard_callbacks]

    # Fit network
    history_callback = NN_model.fit(x=df_train,
                                    y=target,
                                    shuffle=True,
                                    epochs=epoch,
                                    batch_size=256,
                                    validation_split=0.1,
                                    workers=2,
                                    callbacks=callbacks_list)
    file = open("NetworkHistory.txt", "a")

    CSV_Callbacks(history_callback, emissions)
    

def Make_network(in_shape):
    NN_model = Sequential()

    NN_model.add(
        Dense(1024,
              kernel_initializer='RandomNormal',
              input_dim=in_shape,
              activation="relu"))

    NN_model.add(
        Dense(1024, kernel_initializer='RandomNormal', activation="relu"))
    NN_model.add(Dense(1, kernel_initializer='RandomNormal', activation=None))
    NN_model = compile_NN(NN_model)
    NN_model.summary()

    return NN_model
    
def CSV_Callbacks(callbacks, directory):

    f = open("callback_history.csv", "w")
    print(type(callbacks.history))
    for title in callbacks.history:
        print(title)
        ##print(title, end==',')

    history_df = pd.DataFrame.from_dict(callbacks.history)
    history_df.to_csv(
        "{}/callback_history.csv".format(directory), index=False
    )  #MIH: the callback_history file gets moved to the right directory

def learning_schedule(epoch):
    learning_rate = .001 * math.exp(-epoch / 20)

    tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
    return learning_rate


def compile_NN(NN_model):
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        .001, decay_rate=.36, decay_steps=1e5)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    NN_model.compile(loss=tf.keras.losses.MeanAbsolutePercentageError(),
                     optimizer=optimizer,
                     metrics=['mse', 'mae', 'mape'])

    return NN_model

### Predicting from NN

In [38]:
def PredictNN(df2,df1,targetList, inputList,dir):

    df_test = df1.copy(deep='all')
    df_train = df2

    target = [df_train[targetList[0]].to_numpy(), df_train[targetList[1]].to_numpy()]
    df_train.drop(columns=targetList,axis=1,inplace=True)

    NN_model = Make_network(df_train.shape[1])

    file = find_min_weights(dir)
    min_loss = None
    pwd = os.getcwd()

    wights_file = file  # choose the best checkpoint
    NN_model.load_weights(wights_file)  # load it

    NN_model = compile_NN(NN_model)


    predictions = (NN_model.predict(df_train))

    df_train['diffL2'] = target[0]
    df_train['diffL4'] = target[1]
    
    df2['diffL2_predicted'] = predictions[0]
    df2['diffL4_predicted'] = predictions[1]

    write_csv_file(df2, dir)

    if debug_NN_predict == True:
        for col in df1.columns:
            print(col)
    tmp = NN_model.layers[0].get_weights()
    
def find_min_weights(directory):
    pwd = os.getcwd()
    min_loss = None
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('hdf5'):
                splt = file.split('--')
                if min_loss == None:
                    min_loss = float(splt[1][:-5])
                    min_file = os.path.join(root, file)
                elif min_loss > float(splt[1][:-5]):
                    min_loss = float(splt[1][:-5])
                    min_file = os.path.join(root, file)
                # print(file)
                # print(splt[1][:-5])
                # print(float(splt[1][:-5]))
    return min_file


def write_csv_file(df, dir, file_name='results.csv'):
    print("testing the write func:" + dir)
    df.to_csv(os.path.join(dir, file_name), header=True, index=False)
    print("CSV file written")

# Run Test

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM, SimpleRNN, Embedding
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import seaborn as sb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model
try:
    # pydot-ng is a fork of pydot that is better maintained.
    import pydot_ng as pydot
except ImportError:
    # pydotplus is an improved version of pydot
    try:
        import pydotplus as pydot
    except ImportError:
        # Fall back on pydot if necessary.
        try:
            import pydot
        except ImportError:
            pydot = None
import math
from datetime import datetime
from MuDataFrame import *
from scipy import stats

In [41]:
#loading the input data 
mdfo_calib = MuDataFrame("/Volumes/mac_extended/Research/MT/proto1b/data_sets/calibration_new_set_up/calibration_new_set_up.csv")

mdf_calib = mdfo_calib.events_df
#keeping 4by4 events meants events that passed through all 4 detector trays
mdfo_calib.keep4by4Events()

,Unnamed: 0,index,event_num,event_time,deadtime,l1hit,l2hit,l3hit,l4hit,r1hit,...,z_angle,SmallCounter,speed,Run_Num,xx,yy,xx1,yy1,time_of_day,time
0,0,0,0,2021-05-03 12:40:15.812760,0,1.0,1.0,1.0,1.0,1.0,...,13.455446,NaN,0.342978,622,-5.445887,-22.562492,-6.807692,-22.684615,day,0
1,1,1,1,2021-05-03 12:40:16.502963,0,1.0,1.0,1.0,1.0,1.0,...,12.002813,NaN,0.703353,622,-11.762760,-21.506829,-11.538462,-21.784615,day,690203
2,2,2,2,2021-05-03 12:40:16.672924,0,1.0,1.0,1.0,1.0,1.0,...,14.672630,NaN,0.210716,622,35.347122,13.620282,36.676923,14.884615,day,860164
3,3,3,3,2021-05-03 12:40:16.892987,0,1.0,1.0,1.0,1.0,1.0,...,15.284964,NaN,0.259347,622,24.903617,25.826667,27.746154,29.153846,day,1080227
4,4,4,4,2021-05-03 12:40:17.563102,0,1.0,1.0,1.0,1.0,1.0,...,4.253100,NaN,11.038012,622,6.978340,14.482143,7.676923,15.230769,day,1750342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999992,2999992,99992,2999992,2021-05-20 13:18:29.305435,0,1.0,1.0,1.0,1.0,1.0,...,4.471381,NaN,2.208244,656,-24.827750,-14.125268,-25.330769,-14.146154,day,1077344492675
2999993,2999993,99993,2999993,2021-05-20 13:18:29.875548,0,1.0,1.0,1.0,1.0,1.0,...,15.702291,NaN,0.346495,656,13.468468,-38.856563,13.800000,-37.423077,day,1077345062788
2999994,2999994,99994,2999994,2021-05-20 13:18:30.125522,0,1.0,1.0,1.0,1.0,1.0,...,13.684312,NaN,0.276322,656,28.348837,-29.674472,28.130769,-29.230769,day,1077345312762
2999996,2999996,99996,2999996,2021-05-20 13:18:30.965416,0,1.0,1.0,1.0,1.0,1.0,...,6.035804,NaN,0.442759,656,12.892377,-12.244589,13.269231,-13.276923,day,1077346152656


In [42]:
from sklearn.model_selection import train_test_split

all_df = mdfo_calib.events_df
all_df.drop(columns=['Unnamed: 0', 'event_time','Run_Num','time_of_day','SmallCounter','speed','xx','yy'],inplace=True)
df_train_all, df_test_all = train_test_split(all_df, test_size=0.3)
targetList = ["diffL2","diffL4"]
inputList = ['event_num','L1','R1', 'L3', 'R3', 'TopCounter', 'BottomCounter', 'diffL1', 'diffL3', 'sumL1', 'sumL3']

Train the Network

In [ ]:
date = datetime.today().strftime('%Y-%m-%d-%H_%M_%S')
emissions = "Run_{}".format(date)

TrainNN(df_train, df_test, targetList, inputList, emissions, epoch=1)

Test the Network

In [ ]:
PredictNN(df2,df1,targetList, inputList, emissions)